In [34]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/UCI_Credit_Card.csv")

df.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [35]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

Rename Target Column

In [36]:
df.rename(columns={
    'default.payment.next.month': 'DEFAULT'
}, inplace=True)
df['DEFAULT'].value_counts()
df['DEFAULT'].value_counts(normalize=True)


,proportion
DEFAULT,
0,0.7788
1,0.2212


0 (Non-default)

1 (Default)

Drop ID Column

In [37]:
df.drop(columns=['ID'], inplace=True)
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'DEFAULT'],
      dtype='object')

23 features + 1 target

EDUCATION Cleanup

In [38]:
df['EDUCATION'].value_counts().sort_index()

,count
EDUCATION,
0,14
1,10585
2,14030
3,4917
4,123
5,280
6,51


In [39]:
df['EDUCATION'] = df['EDUCATION'].replace([5, 6, 0], 4)
df['EDUCATION'].value_counts().sort_index()

,count
EDUCATION,
1,10585
2,14030
3,4917
4,468


Value	Meaning

1	Graduate

2	University

3	High School

4	Others

MARRIAGE column

Problem:

Value 0 is invalid

In [40]:
df['MARRIAGE'].value_counts().sort_index()

,count
MARRIAGE,
0,54
1,13659
2,15964
3,323


In [41]:
df['MARRIAGE'] = df['MARRIAGE'].replace(0, 3)
df['MARRIAGE'].value_counts().sort_index()

,count
MARRIAGE,
1,13659
2,15964
3,377


Value	Meaning

1	Married

2	Single

3	Others

Check AGE Validity

Finance logic:

Credit cards are not issued to minors

Extreme ages distort model

In [42]:
df['AGE'].describe()

,AGE
count,30000.000000
mean,35.485500
std,9.217904
min,21.000000
25%,28.000000
50%,34.000000
75%,41.000000
max,79.000000


In [43]:
df[df['AGE'] < 18]
df[df['AGE'] > 85]

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT


Age values are within a valid customer range (21–79); no treatment required


Payment Status Columns Sanity Check

In [44]:
pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']

for col in pay_cols:
    print(col, sorted(df[col].unique()))

PAY_0 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_2 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_3 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_4 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_5 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_6 [np.int64(-2), np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]


The original data contained -2, representing no credit usage.
For credit risk modeling, we normalized -2 to -1 to represent non-delinquent behavior, ensuring consistent risk interpretation across time.

In [45]:
pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']

df[pay_cols] = df[pay_cols].replace(-2, -1)

for col in pay_cols:
    print(col, sorted(df[col].unique()))


PAY_0 [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_2 [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_3 [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_4 [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_5 [np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]
PAY_6 [np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]


Check for Missing Values

In [46]:
df.isnull().sum()

,0
LIMIT_BAL,0
SEX,0
EDUCATION,0
MARRIAGE,0
AGE,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0
PAY_5,0


Check Negative Amounts (Bills & Payments)

Columns:

BILL_AMT1–6

PAY_AMT1–6

Logic:

Bill amounts can be negative (credit adjustment)

Payment amounts cannot be negative

In [47]:
bill_cols = [f'BILL_AMT{i}' for i in range(1,7)]
pay_amt_cols = [f'PAY_AMT{i}' for i in range(1,7)]

(df[pay_amt_cols] < 0).sum()

,0
PAY_AMT1,0
PAY_AMT2,0
PAY_AMT3,0
PAY_AMT4,0
PAY_AMT5,0
PAY_AMT6,0


Time Consistency Check

Rule is If someone is 3 months late (PAY_0 = 3), they should not have paid the full bill that month.

In [48]:
df[['PAY_0','BILL_AMT1','PAY_AMT1']].sample(10)

,PAY_0,BILL_AMT1,PAY_AMT1
18416,0,30767.0,1600.0
7790,1,0.0,0.0
25315,0,34183.0,1700.0
17619,2,118656.0,10000.0
16828,0,117255.0,5400.0
23819,1,-9.0,1093.0
1160,0,70524.0,2500.0
8324,-1,10228.0,50836.0
9374,0,48566.0,2000.0
14552,-1,390.0,390.0


In [49]:
pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']

df['MAX_DELAY'] = df[pay_cols].max(axis=1)
df['AVG_DELAY'] = df[pay_cols].mean(axis=1)

# Trend: recent - oldest
df['DELINQUENCY_TREND'] = df['PAY_0'] - df['PAY_6']

df[['MAX_DELAY','AVG_DELAY','DELINQUENCY_TREND']].describe()


,MAX_DELAY,AVG_DELAY,DELINQUENCY_TREND
count,30000.000000,30000.000000,30000.000000
mean,0.509033,-0.046800,0.203200
std,1.237701,0.814206,1.024538
min,-1.000000,-1.000000,-7.000000
25%,0.000000,-0.666667,0.000000
50%,0.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000
max,8.000000,6.000000,6.000000


MAX_DELAY - The maximum number of months late the customer was in the last 6 months.

Mean: 0.51

Median (50%): 0 - 50% of customers were never late

75%: 2 - 75% were late no more than 2 months

Max: 8 - Some customers went 8 months late → very high risk

📌 MAX_DELAY is a strong red-flag feature
Risk teams care more about worst behavior than average behavior.

AVG_DELAY - Average delinquency over 6 months.

Mean: -0.0468 - Negative values → mostly on-time / no usage

Median: 0 - typical customer pays on time

Min: -1

Max: 6

📌 AVG_DELAY captures consistency

Someone late once ≠ someone late always

DELINQUENCY_TREND

Mean: 0.20

Median: 0

Min: -7

Max: 6

Most customers are stable

Some are rapidly deteriorating → prime intervention targets

Negative extremes = customers recovering

📌 Trend features are gold

They catch risk before default happens.

A customer with:

MAX_DELAY ≥ 3

AVG_DELAY > 1

DELINQUENCY_TREND > 0

➡️ High probability of default

In [50]:
#Create a High-Risk Flag
df['HIGH_RISK_RULE'] = (
    (df['MAX_DELAY'] >= 3) &
    (df['AVG_DELAY'] > 1) &
    (df['DELINQUENCY_TREND'] > 0)
).astype(int)

#Count How Many Customers Match
df['HIGH_RISK_RULE'].value_counts()


,count
HIGH_RISK_RULE,
0,29555
1,445


This gives:

1 → customers matching the risky pattern

0 → everyone else

Utilization captures how aggressively customers use their available credit.

In [51]:
df['AVG_BILL'] = df[bill_cols].mean(axis=1)
df['UTILIZATION'] = df['AVG_BILL'] / df['LIMIT_BAL']
df['UTILIZATION'] = df['UTILIZATION'].clip(0, 1.5)

Payment Behavior Ratio

In [52]:
df['TOTAL_BILL'] = df[bill_cols].sum(axis=1)
df['TOTAL_PAY'] = df[pay_amt_cols].sum(axis=1)

df['PAYMENT_RATIO'] = df['TOTAL_PAY'] / (df['TOTAL_BILL'] + 1)

Percentage of Portfolio

In [53]:
df['HIGH_RISK_RULE'].mean()

np.float64(0.014833333333333334)

Only ~1.5% of the portfolio shows this extreme risky behavior.

In [54]:
#How Bad Are They? (Validation)
pd.crosstab(
    df['HIGH_RISK_RULE'],
    df['DEFAULT'],
    normalize='index'
)


DEFAULT,0,1
HIGH_RISK_RULE,,
0,0.785383,0.214617
1,0.341573,0.658427


Default rates by group

Group -	Default Rate

LOW / NORMAL RISK	- 21.46%

HIGH RISK (Rule = 1)	- 65.84%

BEHAVIOR STABILITY (Volatility)

In [55]:
df['DELAY_STD'] = df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].std(axis=1)
df['DELAY_STD'].describe()

,DELAY_STD
count,30000.000000
mean,0.394847
std,0.477574
min,0.000000
25%,0.000000
50%,0.000000
75%,0.816497
max,3.204164


What is DELAY_STD?

Standard deviation of payment delays over 6 months

Measures how stable or erratic a customer’s payment behavior is

Captures behavioral inconsistency, not just lateness

This is a key differentiator between chronic defaulters and accidental late payers.

Metric - Value - Meaning
Mean -	0.39 -	Most customers are fairly stable

Median -	0.00 -	Over 50% have no variation (always same status)

75% -	0.82 -	Only top 25% show noticeable variability

Max -	3.20 -	Highly erratic behavior

What this tells us:

Majority of customers:

Either always pay on time

Or always revolve at same level

Small group:

Alternates between on-time and delinquent

Much harder to predict → higher risk

📌 High DELAY_STD = unstable behavior = red flag

Average Bill & Payment

In [56]:
bill_cols = [f'BILL_AMT{i}' for i in range(1,7)]
pay_cols_amt = [f'PAY_AMT{i}' for i in range(1,7)]

df['AVG_BILL_AMT'] = df[bill_cols].mean(axis=1)
df['AVG_PAY_AMT'] = df[pay_cols_amt].mean(axis=1)


Credit Utilization

In [57]:
df['CREDIT_UTILIZATION'] = df['AVG_BILL_AMT'] / df['LIMIT_BAL']
df['CREDIT_UTILIZATION'] = df['CREDIT_UTILIZATION'].clip(0, 1.5)

Payment Ratio

Does the customer pay what they owe?

In [58]:
df['PAYMENT_RATIO'] = np.where(
    df['AVG_BILL_AMT'] > 0,
    df['AVG_PAY_AMT'] / df['AVG_BILL_AMT'],
    np.nan
)
#Cap extreme values (winsorization)
df['PAYMENT_RATIO'] = df['PAYMENT_RATIO'].clip(lower=0, upper=5)


In [59]:
df[['PAYMENT_RATIO']].describe()

,PAYMENT_RATIO
count,28929.000000
mean,0.384064
std,0.597153
min,0.000000
25%,0.042515
50%,0.094530
75%,0.622939
max,5.000000


In [60]:
df['PAYMENT_RATIO'].isna().sum()

np.int64(1071)

In [61]:
df['PAYMENT_RATIO'] = df['PAYMENT_RATIO'].fillna(1)

Customers with zero outstanding balance were treated as fully paid, since no repayment obligation exists.

In [62]:
features = [
    'LIMIT_BAL',
    'CREDIT_UTILIZATION',
    'PAYMENT_RATIO',
    'MAX_DELAY',
    'AVG_DELAY',
    'DELINQUENCY_TREND',
    'DELAY_STD',
    'AGE',
    'SEX',
    'EDUCATION',
    'MARRIAGE'
]

X = df[features]
y = df['DEFAULT']

X.describe()

,LIMIT_BAL,CREDIT_UTILIZATION,PAYMENT_RATIO,MAX_DELAY,AVG_DELAY,DELINQUENCY_TREND,DELAY_STD,AGE,SEX,EDUCATION,MARRIAGE
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,0.372429,0.406053,0.509033,-0.046800,0.203200,0.394847,35.485500,1.603733,1.842267,1.557267
std,129747.661567,0.348053,0.597429,1.237701,0.814206,1.024538,0.477574,9.217904,0.489129,0.744494,0.521405
min,10000.000000,0.000000,0.000000,-1.000000,-1.000000,-7.000000,0.000000,21.000000,1.000000,1.000000,1.000000
25%,50000.000000,0.029997,0.043163,0.000000,-0.666667,0.000000,0.000000,28.000000,1.000000,1.000000,1.000000
50%,140000.000000,0.284834,0.106452,0.000000,0.000000,0.000000,0.000000,34.000000,2.000000,2.000000,2.000000
75%,240000.000000,0.687929,0.718489,2.000000,0.000000,0.000000,0.816497,41.000000,2.000000,2.000000,2.000000
max,1000000.000000,1.500000,5.000000,8.000000,6.000000,6.000000,3.204164,79.000000,2.000000,4.000000,3.000000


Train test split

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1. Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

# 2. Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Logistic Regression
log_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    solver='lbfgs'
)

log_model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [71]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

y_pred = log_model.predict(X_test_scaled)
y_proba = log_model.predict_proba(X_test_scaled)[:, 1]

roc_auc_score(y_test, y_proba)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[5393 1616]
 [ 769 1222]]
              precision    recall  f1-score   support

           0       0.88      0.77      0.82      7009
           1       0.43      0.61      0.51      1991

    accuracy                           0.73      9000
   macro avg       0.65      0.69      0.66      9000
weighted avg       0.78      0.73      0.75      9000



ROC–AUC = 0.733.

The model can rank a random defaulter above a random non-defaulter ~73% of the time

Default Class (1) — THIS IS THE FOCUS

Recall (Sensitivity): 0.61

Precision: 0.43

F1-score: 0.51

The model correctly identifies ~61% of defaulters, prioritizing risk capture over precision.

Non-Default Class (0)

Precision: 0.88

Recall: 0.77

ow-risk customers are largely preserved, limiting unnecessary adverse actions.

false negatives are more expensive than false positives:


False Negative (769) → Missed defaulters → financial loss

False Positive (1616) → Extra monitoring / limit review → manageable cost

In [72]:
y_pred_proba = log_model.predict_proba(X_test_scaled)[:, 1]

In [74]:
threshold = 0.35
y_pred_custom = (y_pred_proba >= threshold).astype(int)
print(classification_report(y_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.89      0.42      0.57      7009
           1       0.28      0.81      0.42      1991

    accuracy                           0.50      9000
   macro avg       0.59      0.62      0.49      9000
weighted avg       0.75      0.50      0.53      9000



In [ ]:
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': log_model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

coef_df


,Feature,Coefficient
4,AVG_DELAY,0.525492
3,MAX_DELAY,0.292769
5,DELINQUENCY_TREND,0.244503
6,DELAY_STD,0.213076
2,PAYMENT_RATIO,0.093030
7,AGE,0.011033
0,LIMIT_BAL,-0.000002
9,EDUCATION,-0.077242
8,SEX,-0.094693
10,MARRIAGE,-0.122258


POSITIVE COEFFICIENTS → ↑ DEFAULT RISK

AVG_DELAY (+): Customers with consistently delayed payments across months have a significantly higher likelihood of default.

MAX_DELAY (+): A history of severe missed payments materially increases default risk.

DELINQUENCY_TREND (+): Worsening recent payment behavior compared to earlier months raises default probability.

DELAY_STD (+): High volatility or inconsistency in repayment behavior is associated with increased credit risk.

PAYMENT_RATIO (+): After accounting for delinquency, partial repayment behavior shows limited incremental reduction in default risk.

AGE (+ ~0): Age has minimal impact on default risk once behavioral factors are considered.

LIMIT_BAL (− ~0): Higher credit limits are weakly associated with lower default risk due to prior risk screening.

EDUCATION (−): Higher education levels correlate slightly with lower default probability.

SEX (−): Gender shows a minor association with default risk compared to behavioral drivers.

MARRIAGE (−): Married customers exhibit marginally lower default risk after controlling for behavior.

CREDIT_UTILIZATION (−): Higher utilization, when paired with timely payments, reflects active but responsible credit usage rather than risk.



In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_features = [
    'LIMIT_BAL',
    'CREDIT_UTILIZATION',
    'PAYMENT_RATIO',
    'MAX_DELAY',
    'AVG_DELAY',
    'DELINQUENCY_TREND',
    'DELAY_STD',
    'AGE'
]

cat_features = ['SEX', 'EDUCATION', 'MARRIAGE']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(drop='first'), cat_features)
    ]
)


In [78]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])


In [79]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['LIMIT_BAL',
                                                   'CREDIT_UTILIZATION',
                                                   'PAYMENT_RATIO', 'MAX_DELAY',
                                                   'AVG_DELAY',
                                                   'DELINQUENCY_TREND',
                                                   'DELAY_STD', 'AGE']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('classifier',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [81]:
from sklearn.metrics import roc_auc_score, classification_report

# Predict using Logistic Regression pipeline
y_pred_lr = model.predict(X_test)
y_pred_lr_proba = model.predict_proba(X_test)[:, 1]

print("Logistic Regression ROC-AUC:", roc_auc_score(y_test, y_pred_lr_proba))
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))


Logistic Regression ROC-AUC: 0.7297345115367563
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82      7009
           1       0.43      0.61      0.51      1991

    accuracy                           0.74      9000
   macro avg       0.65      0.69      0.66      9000
weighted avg       0.78      0.74      0.75      9000



ROC–AUC: 0.7297

Interpretation

Logistic Regression ROC–AUC ≈ 0.73



In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report

dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(
        max_depth=5,              # keeps tree interpretable
        min_samples_leaf=100,     # avoids overfitting
        class_weight='balanced',
        random_state=42
    ))
])


In [83]:
dt_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['LIMIT_BAL',
                                                   'CREDIT_UTILIZATION',
                                                   'PAYMENT_RATIO', 'MAX_DELAY',
                                                   'AVG_DELAY',
                                                   'DELINQUENCY_TREND',
                                                   'DELAY_STD', 'AGE']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('classifier',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=5,
                                        min_samples_leaf=100,
                                        random_state=42))])

In [84]:
y_pred_dt = dt_pipeline.predict(X_test)
y_pred_dt_proba = dt_pipeline.predict_proba(X_test)[:, 1]

print("Decision Tree ROC-AUC:", roc_auc_score(y_test, y_pred_dt_proba))
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))


Decision Tree ROC-AUC: 0.7616833175455908
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.76      0.82      7009
           1       0.43      0.64      0.51      1991

    accuracy                           0.73      9000
   macro avg       0.66      0.70      0.67      9000
weighted avg       0.78      0.73      0.75      9000



Random Forest

In [85]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=8,
        min_samples_leaf=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])


In [86]:
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['LIMIT_BAL',
                                                   'CREDIT_UTILIZATION',
                                                   'PAYMENT_RATIO', 'MAX_DELAY',
                                                   'AVG_DELAY',
                                                   'DELINQUENCY_TREND',
                                                   'DELAY_STD', 'AGE']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=8,
                                        min_samples_leaf=100, n_estimators=300,
                                        n_jobs=-1, random_state=42))])

In [87]:
y_pred_rf = rf_pipeline.predict(X_test)
y_pred_rf_proba = rf_pipeline.predict_proba(X_test)[:, 1]

print("Random Forest ROC-AUC:", roc_auc_score(y_test, y_pred_rf_proba))
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest ROC-AUC: 0.7706567125183601
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82      7009
           1       0.44      0.62      0.51      1991

    accuracy                           0.74      9000
   macro avg       0.66      0.70      0.67      9000
weighted avg       0.78      0.74      0.75      9000



Soft Voting Ensemble

In [88]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report


In [89]:
lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])

dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(
        max_depth=5,
        min_samples_leaf=100,
        class_weight='balanced',
        random_state=42
    ))
])

rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=8,
        min_samples_leaf=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])


Voting Ensemble

In [90]:
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('dt', dt),
        ('rf', rf)
    ],
    voting='soft'   # probability averaging
)


Train & Evaluate

In [91]:
voting_clf.fit(X_train, y_train)

y_pred_vote = voting_clf.predict(X_test)
y_pred_vote_proba = voting_clf.predict_proba(X_test)[:, 1]

print("Voting Ensemble ROC-AUC:", roc_auc_score(y_test, y_pred_vote_proba))
print(classification_report(y_test, y_pred_vote))


Voting Ensemble ROC-AUC: 0.7680617852385958
              precision    recall  f1-score   support

           0       0.87      0.79      0.83      7009
           1       0.45      0.60      0.51      1991

    accuracy                           0.75      9000
   macro avg       0.66      0.70      0.67      9000
weighted avg       0.78      0.75      0.76      9000



Stacking Ensemble

In [92]:
from sklearn.ensemble import StackingClassifier

Stacking with Logistic meta-model

In [93]:
stack_clf = StackingClassifier(
    estimators=[
        ('lr', lr),
        ('dt', dt),
        ('rf', rf)
    ],
    final_estimator=LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ),
    stack_method='predict_proba',
    passthrough=False
)


Train & Evaluate

In [94]:
stack_clf.fit(X_train, y_train)

y_pred_stack = stack_clf.predict(X_test)
y_pred_stack_proba = stack_clf.predict_proba(X_test)[:, 1]

print("Stacking ROC-AUC:", roc_auc_score(y_test, y_pred_stack_proba))
print(classification_report(y_test, y_pred_stack))


Stacking ROC-AUC: 0.7715278032068836
              precision    recall  f1-score   support

           0       0.88      0.78      0.83      7009
           1       0.45      0.62      0.52      1991

    accuracy                           0.75      9000
   macro avg       0.66      0.70      0.67      9000
weighted avg       0.78      0.75      0.76      9000



In [95]:
models = {
    "Logistic Regression": model,          # LR pipeline
    "Decision Tree": dt_pipeline,
    "Random Forest": rf_pipeline,
    "Voting Ensemble": voting_clf,
    "Stacking Ensemble": stack_clf
}
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

def evaluate_models(models, X_test, y_test):
    results = []

    for name, mdl in models.items():
        y_pred = mdl.predict(X_test)
        y_proba = mdl.predict_proba(X_test)[:, 1]

        results.append({
            "Model": name,
            "ROC_AUC": roc_auc_score(y_test, y_proba),
            "Default_Recall": recall_score(y_test, y_pred, pos_label=1),
            "Default_Precision": precision_score(y_test, y_pred, pos_label=1),
            "Accuracy": accuracy_score(y_test, y_pred)
        })

    return pd.DataFrame(results)
comparison_df = evaluate_models(models, X_test, y_test)

comparison_df.sort_values(by="ROC_AUC", ascending=False)


,Model,ROC_AUC,Default_Recall,Default_Precision,Accuracy
4,Stacking Ensemble,0.771528,0.618785,0.448163,0.747111
2,Random Forest,0.770657,0.618282,0.436215,0.738778
3,Voting Ensemble,0.768062,0.601205,0.447477,0.747556
1,Decision Tree,0.761683,0.636364,0.432423,0.734778
0,Logistic Regression,0.729735,0.614766,0.431594,0.735667


Tree-based and ensemble models provided marginal ROC-AUC improvements; however, logistic regression was retained as the primary model due to interpretability, with decision trees used as challengers for policy insights.

In [96]:
comparison_df.style.highlight_max(
    subset=["ROC_AUC", "Default_Recall", "Accuracy"],
    color="lightgreen"
)

,Model,ROC_AUC,Default_Recall,Default_Precision,Accuracy
0,Logistic Regression,0.729735,0.614766,0.431594,0.735667
1,Decision Tree,0.761683,0.636364,0.432423,0.734778
2,Random Forest,0.770657,0.618282,0.436215,0.738778
3,Voting Ensemble,0.768062,0.601205,0.447477,0.747556
4,Stacking Ensemble,0.771528,0.618785,0.448163,0.747111


In [97]:
import joblib

joblib.dump(model, "credit_risk_logistic_pipeline.pkl")


['credit_risk_logistic_pipeline.pkl']

In [98]:
test_case_2 = {
    "LIMIT_BAL": 70000,
    "CREDIT_UTILIZATION": 0.75,
    "PAYMENT_RATIO": 0.45,
    "MAX_DELAY": 2,
    "AVG_DELAY": 1.5,
    "DELINQUENCY_TREND": 1,
    "DELAY_STD": 1.2,
    "AGE": 42,
    "SEX": 1,          # Male
    "EDUCATION": 2,    # University
    "MARRIAGE": 2      # Single
}

df_test = pd.DataFrame([test_case_2])

prob = model.predict_proba(df_test)[0][1]

print(f"Default Probability: {prob:.2%}")

Default Probability: 85.19%
